# Simulate an EPICS motor

Using two float PVs and a userCalc from the docker container, simulate an EPICS motor with tools from ophyd and apstools.

userCalc field | value | meaning
---  | ---          | ---
A PV | gp:gp:float1 | motor VAL
B PV | gp:gp:float2 | motor RBV
C    | 0.1          | maximum step size
CALC | `MIN(ABS(A-B),C)*(A>B?1:-1)+B` | simulated position (readback)
SCAN | 1 second     | update rate (motor speed)
OUTN | gp:gp:float2 | output PV name

In [1]:
from apstools.devices import PVPositionerSoftDone
from apstools.devices import SwaitRecord

import bluesky
from bluesky import plans as bp
from bluesky import plan_stubs as bps

from ophyd import Component
from ophyd import EpicsSignal
from ophyd import PVPositionerPC

import time

In [2]:
class SoftMotor(PVPositionerPC):
    readback = Component(EpicsSignal, "gp:float2")
    setpoint = Component(EpicsSignal, "gp:float1")

In [3]:
motor = SoftMotor("gp:", name="motor")
motor.wait_for_connection()

In [4]:
sim_calc = SwaitRecord("gp:userCalc5", name="sim_calc")
sim_calc.wait_for_connection()

In [5]:
sim_calc.reset()
sim_calc
sim_calc.channels.A.input_pv.put(motor.setpoint.pvname)
sim_calc.channels.B.input_pv.put(motor.readback.pvname)
sim_calc.channels.C.input_value.put(0.1)
sim_calc.scanning_rate.put(".1 second")
sim_calc.calculation.put("MIN(ABS(A-B),C)*(A>B?1:-1)+B")
sim_calc.output_link_pv.put(motor.readback.pvname)
sim_calc.description.put(f"{motor.name} (simulated)")

In [6]:
sim_calcs_enable = EpicsSignal("gp:userCalcEnable", name="sim_calcs_enable")
sim_calcs_enable.wait_for_connection()
sim_calcs_enable.put("Enable")

In [7]:
motor.setpoint.put(1.5+motor.position)

In [8]:
RE = bluesky.RunEngine({})

In [9]:
t0 = time.time()
RE(bps.mvr(motor, 2))
print(f"{time.time()-t0:.3f}s elapsed")

0.009s elapsed


In [10]:
motor = PVPositionerSoftDone("gp:", readback_pv="gp:float2", setpoint_pv="gp:float1", tolerance=0.001, name="motor")
motor.wait_for_connection()

In [11]:
motor.move(1.234)

MoveStatus(done=True, pos=motor, elapsed=1.0, success=True, settle_time=0.0)

In [13]:
t0 = time.time()
RE(bps.mv(motor, -motor.position))
print(f"{time.time()-t0:.3f}s elapsed")

2.454s elapsed
